# Christmas Data Science Competition

Author: Hans-Jörg Stark   
Date: December 2021

In [1]:
#Load some modules first
import os, sys
import pandas as pd
import numpy as np

In [4]:
#load the data
path='/Users/hansjoerg.stark/Library/Mobile Documents/com~apple~CloudDocs/Privat/Weiterbildung/Bethel-DataScience/_Projects/xmasChallenge/Data'
rawDataFl = os.path.join(path,'christmas_billboard_data.csv')
dfXmasSongsRaw = pd.read_csv(rawDataFl)
dfXmasSongsRaw.head(7)


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,month,day
0,http://www.billboard.com/charts/hot-100/1958-1...,12/13/1958,83,RUN RUDOLPH RUN,Chuck Berry,Run Rudolph RunChuck Berry,1,NaN,69,3,1958,12,13
1,http://www.billboard.com/charts/hot-100/1958-1...,12/20/1958,57,JINGLE BELL ROCK,Bobby Helms,Jingle Bell RockBobby Helms,1,NaN,29,19,1958,12,20
2,http://www.billboard.com/charts/hot-100/1958-1...,12/20/1958,73,RUN RUDOLPH RUN,Chuck Berry,Run Rudolph RunChuck Berry,1,83.0,69,3,1958,12,20
3,http://www.billboard.com/charts/hot-100/1958-1...,12/20/1958,86,WHITE CHRISTMAS,Bing Crosby,White ChristmasBing Crosby,1,NaN,12,13,1958,12,20
4,http://www.billboard.com/charts/hot-100/1958-1...,12/27/1958,44,GREEN CHRI$TMA$,Stan Freberg,Green Chri$tma$Stan Freberg,1,NaN,44,2,1958,12,27
5,http://www.billboard.com/charts/hot-100/1958-1...,12/27/1958,66,WHITE CHRISTMAS,Bing Crosby,White ChristmasBing Crosby,1,86.0,12,13,1958,12,27
6,http://www.billboard.com/charts/hot-100/1958-1...,12/27/1958,69,RUN RUDOLPH RUN,Chuck Berry,Run Rudolph RunChuck Berry,1,73.0,69,3,1958,12,27


In [6]:
#get some basic information on the dataset, especially data-types
dfXmasSongsRaw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   url                     387 non-null    object 
 1   weekid                  387 non-null    object 
 2   week_position           387 non-null    int64  
 3   song                    387 non-null    object 
 4   performer               387 non-null    object 
 5   songid                  387 non-null    object 
 6   instance                387 non-null    int64  
 7   previous_week_position  279 non-null    float64
 8   peak_position           387 non-null    int64  
 9   weeks_on_chart          387 non-null    int64  
 10  year                    387 non-null    int64  
 11  month                   387 non-null    int64  
 12  day                     387 non-null    int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 39.4+ KB
